In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import numba
from numba import jit, njit, prange, int64, cuda
from tqdm import tqdm_notebook as tqdm

In [3]:
from TimeIntegrate import tvd_rk3
from SimSettings import Prim, Gas, Sim, Cons
from BCs import apply_bcs
from PrimCons import prim2cons, prim2flux, cons2prim
from Weno import reconstruct_weno

/Users/goobley/NewImex/NumbaTime/Weno.py:42: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  enoIntpL[s] = np.dot(q[row, gridIdx:gridIdx+3], EnoCoeffL[2-s])
/Users/goobley/NewImex/NumbaTime/Weno.py:43: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  enoIntpR[s] = np.dot(q[row, gridIdx:gridIdx+3], EnoCoeffR[s])


In [4]:
def cfl(V):
    cs = np.sqrt(Gas.Gamma * V[Prim.Pres] / V[Prim.Dens])
    vel = cs + np.abs(V[Prim.Velo])
    dt = 0.9 * Sim.GrDx / np.max(vel)
    return dt

In [5]:
def driver(V, U, tMax=None):
    dt = cfl(V)
    if tMax is not None:
        dt = min(dt, tMax)
    V[:], U[:] = tvd_rk3(dt, V, U)
    apply_bcs(V)
    return dt
    

In [6]:
def setup():
    mid = (Sim.GriBeg + Sim.GriEnd) // 2
    V = np.zeros((Prim.NumVars, Sim.GriMax))
    
    V[Prim.Dens, :mid] = 1.0
    V[Prim.Pres, :mid] = 1.0
    V[Prim.Velo] = 0.0
    V[Prim.Dens, mid:] = 0.125
    V[Prim.Pres, mid:] = 0.1
    V[Prim.Eint] = V[Prim.Pres] / ((Gas.Gamma-1.0) * V[Prim.Dens])
    
#     mid = int((griBeg + griEnd) * 2/ 5)
#     gr_V[DENS, :mid] = 10.0
#     gr_V[PRES, :mid] = 100.0
#     gr_V[VEL] = 0.0
#     gr_V[DENS, mid:] = 1.0
#     gr_V[PRES, mid:] = 1.0
#     gr_V[EINT] = gr_V[PRES] / ((Gamma-1.0) * gr_V[DENS])
#     gr_V[GAMC] = Gamma
#     gr_V[GAME] = Gamma

#     mid = int((griBeg + griEnd) * 0.1)
#     gr_V[DENS, :mid] = 3.857143
#     gr_V[VEL, :mid] = 2.629369
#     gr_V[PRES, :mid] = 10.3333333
#     gr_V[DENS, mid:] = 1 + 0.2 * np.sin(5 * gridPos[mid:])
#     gr_V[VEL, mid:] = 0.0
#     gr_V[PRES, mid:] = 1.0
#     gr_V[EINT] = gr_V[PRES] / ((Gamma-1.0) * gr_V[DENS])
#     gr_V[GAMC] = Gamma
#     gr_V[GAME] = Gamma

#     gr_V[DENS, :] = 1.0
#     gr_V[VEL, :mid] = -2.0
#     gr_V[VEL, mid:] = 2.0
#     gr_V[EINT, :] = 3.0
#     gr_V[PRES, :] = gr_V[EINT] * (Gamma-1.0) * gr_V[DENS]
#     gr_V[GAMC] = Gamma
#     gr_V[GAME] = Gamma

#     H = 0.5*xEnd # = cs2/|g|
#     cs2 = H * np.abs(Grav)
#     gr_V[PRES, :] = 1e7 * np.exp(-gridPos / H)
#     gr_V[DENS, :] = 1.0 / cs2 * gr_V[PRES, :] 
#     gr_V[VEL, :] = 0.0
#     gr_V[EINT, :] = gr_V[PRES] / ((Gamma-1.0) * gr_V[DENS])
#     gr_V[GAMC] = Gamma
#     gr_V[GAME] = Gamma
    
    apply_bcs(V)
    U = prim2cons(V)
    
    return V, U

In [10]:
# def foo():
V, U = setup()
Vinit = np.copy(V)
t=0
tEnd = 0.2
vs = [Vinit]
tq = tqdm(range(100000))
for i in tq:
    tq.set_description('%f s' % t)
    tMax = tEnd - t
    t += driver(V, U, tMax)
    vs.append(np.copy(V))
    if t >= tEnd:
        break
print(t)

0.2


In [8]:
foo()

0.2


In [9]:
%timeit foo()

0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
30.2 s ± 408 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
plt.figure()
plt.plot(Sim.XPos, vs[-1][Prim.Dens])
# plt.plot(vs[-1][Prim.Pres])
# plt.plot(vs[-1][Prim.Eint])

In [ ]:
V, U = setup()

In [ ]:
reconstruct_weno(V)

In [ ]:
np.seterr(all='raise')
def lax_friedrichs_flux(primRecon):
    consRecon = np.zeros((Cons.NumVars, primRecon.shape[1], primRecon.shape[2]))
    fluxLR = np.zeros_like(consRecon)
    fluxLR[:, 0, :] = prim2flux(primRecon[:, 0, :])
    fluxLR[:, 1, :] = prim2flux(primRecon[:, 1, :])
    consRecon[:, 0, :] = prim2cons(primRecon[:, 0, :])
    consRecon[:, 1, :] = prim2cons(primRecon[:, 1, :])

    csL = np.sqrt(Gas.Gamma * primRecon[Prim.Pres, 0] / primRecon[Prim.Dens, 0])
    csR = np.sqrt(Gas.Gamma * primRecon[Prim.Pres, 1] / primRecon[Prim.Dens, 1])
    maxC = np.zeros_like(csL)
    # The left hand sound speed is the right hand extrapolation of the cell below,
    # and the left hand one is the right hand extrapolation of the value in the
    # next cell
    maxC[1:] = 0.5 * (csL[:-1] + np.abs(primRecon[Prim.Velo, 1, :-1]) + csR[1:] + np.abs(primRecon[Prim.Velo, 0, 1:]))
    # maxC[1:] = 0.5 * (csL[1:] + np.abs(primRecon[Prim.Velo, 0, 1:]) + csL[:-1] + np.abs(primRecon[Prim.Velo, 0, :-1]))

    flux = np.zeros((Cons.NumVars, primRecon.shape[2]))
    flux[:, 1:] = 0.5 * (fluxLR[:, 1, :-1] + fluxLR[:, 0, 1:] - maxC[1:] * (consRecon[:, 1, 1:] - consRecon[:, 0, :-1]))
    return flux

In [ ]:
plt.figure()
plt.plot(Sim.XPos, V[Cons.Dens])
plt.plot(Sim.XPos - Sim.GrDx/2, reconstruct_weno(V)[Cons.Dens, 0], '+')
plt.plot(Sim.XPos + Sim.GrDx/2, reconstruct_weno(V)[Cons.Dens, 1], '+')

In [ ]:
from Flux import lax_friedrichs_flux